In [1]:
import numpy as np
import pandas as pd
import os
import sklearn

# Reading Data

In [2]:
def segmentWords(s): 
    return s.split()

def readFile(fileName):
    # Function for reading file
    # input: filename as string
    # output: contents of file as list containing single words
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = segmentWords('\n'.join(contents))
    return result

#### Create a Dataframe containing the counts of each word in a file

In [115]:
d = []

for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        words = readFile(directory + "/" + file)
        e = {x:words.count(x) for x in words}
        e['__FileID__'] = file
        e['__CLASS__'] = c
        d.append(e)
d = d[:len(d)//20] + d[19*len(d)//20:]

Create a dataframe from d - make sure to fill all the nan values with zeros.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html


In [116]:
df = pd.DataFrame(data=d)
df = df.fillna(value=0)

In [117]:
print(df[:5])

     !     "   #1   #2  #@%$^  $1000  $20  $200  $21  $25    ...      zone  \
0  3.0  10.0  0.0  0.0    0.0    0.0  0.0   0.0  0.0  0.0    ...       0.0   
1  0.0   2.0  0.0  0.0    0.0    0.0  0.0   0.0  0.0  0.0    ...       0.0   
2  0.0   0.0  0.0  0.0    0.0    0.0  0.0   0.0  0.0  0.0    ...       0.0   
3  0.0  20.0  0.0  0.0    0.0    0.0  0.0   0.0  0.0  0.0    ...       0.0   
4  0.0   4.0  0.0  0.0    0.0    0.0  0.0   0.0  0.0  0.0    ...       0.0   

   zoologist  zoom-ins  zoom-outs  zorro  zuko  zurg's  zwick  zwick's  \
0        0.0       0.0        0.0    0.0   0.0     0.0    0.0      0.0   
1        0.0       0.0        0.0    0.0   0.0     0.0    0.0      0.0   
2        0.0       0.0        0.0    0.0   0.0     0.0    0.0      0.0   
3        0.0       0.0        0.0    0.0   0.0     0.0    0.0      0.0   
4        0.0       0.0        0.0    0.0   0.0     0.0    0.0      0.0   

   zwigoff's  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.

#### Split data into training and validation set 

* Sample 80% of your dataframe to be the training data

* Let the remaining 20% be the validation data (you can filter out the indicies of the original dataframe that weren't selected for the training data)

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [126]:
training = df.sample(frac=.8)
#cant figure out validation...
validation = df
for thing in list(training.index.values):
    validation = validation.drop(thing)


* Split the dataframe for both training and validation data into x and y dataframes - where y contains the labels and x contains the words

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [127]:
y_train = training['__CLASS__']
x_train = training.drop('__CLASS__', 1, inplace=True)
x_train = training.drop('__FileID__', 1)
y_numbers = []
for elem in y_train:
    y_numbers.append(int(elem == 'neg'))


y_valid = y_train
x_valid = validation
try:
    x_valid = validation.drop('__CLASS__', 1, inplace=True)
    x_valid = validation.drop('__FileID__', 1)
except KeyError:
    pass
y_numbersv = []
for elem in y_valid:
    y_numbersv.append(int(elem == 'neg'))


# Logistic Regression

#### Basic Logistic Regression
* Use sklearn's linear_model.LogisticRegression() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [128]:
import sklearn.linear_model
model = sklearn.linear_model.LogisticRegression()
model.fit(x_train, y_numbers)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Changing Parameters

#### Feature Selection
* In the backward stepsize selection method, you can remove coefficients and the corresponding x columns, where the coefficient is more than a particular amount away from the mean - you can choose how far from the mean is reasonable.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html

How did you select which features to remove? Why did that reduce overfitting?

# Single Decision Tree

#### Basic Decision Tree

* Initialize your model as a decision tree with sklearn.
* Fit the data and labels to the model.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


In [129]:
from sklearn.tree import DecisionTreeClassifier

stanford_mascot = DecisionTreeClassifier() #no max depth, no max features, no max leaves
stanford_mascot.fit(x_train, y_train) #fitting features to labels
train_score = stanford_mascot.score(x_train, y_train) #accuracy score for the training set, expected to be high

validation_score = stanford_mascot.score(x_valid, y_valid) #accuracy score for the validation set, preparing to disown

ValueError: Found input variables with inconsistent numbers of samples: [128, 32]

#### Changing Parameters
* To test out which value is optimal for a particular parameter, you can either loop through various values or look into sklearn.model_selection.GridSearchCV

References:


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [130]:
from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(estimator = stanford_mascot, param_grid = x_train, refit = True)
gridsearch.fit(x_train, y_train)
print(gridsearch.best_params_)

oski_tree_slayer = gridsearch.best_estimator_
oski_train = oski_tree_slayer.score(x_train, y_train)
oski_valid = oski_tree_slayer.score(x_valid, y_valid)

ValueError: Parameter values for parameter (!) need to be a sequence(but not a string) or np.ndarray.

How did you choose which parameters to change and what value to give to them? Feel free to show a plot.

Gridsearch power! As you can see from above, the best params are blahblahblah will edit later

Why is a single decision tree so prone to overfitting?

It sucks? jk read slides later

# Random Forest Classifier

#### Basic Random Forest

* Use sklearn's ensemble.RandomForestClassifier() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


#### Changing Parameters

What parameters did you choose to change and why?

How does a random forest classifier prevent overfitting better than a single decision tree?